In [31]:
import pandas as pd
import networkx as nx
import itertools
import random

In [47]:
G_reddit = pd.read_csv('reddit.tsv', delimiter="\t")
G_reddit = G_reddit.sort_values('TIMESTAMP')
G_reddit = G_reddit.drop(labels=["POST_ID","TIMESTAMP","PROPERTIES"], axis=1)
# G_reddit = G_reddit.groupby(by=["SOURCE_SUBREDDIT","TARGET_SUBREDDIT"]).sum()
train = G_reddit.iloc[:1000]
test  = G_reddit.iloc[1000:]
train.to_csv('train.csv')
test.to_csv('test.csv')

In [48]:
G_train = nx.convert_matrix.from_pandas_edgelist(train, source="SOURCE_SUBREDDIT", target="TARGET_SUBREDDIT", edge_attr="LINK_SENTIMENT")
G_test = nx.convert_matrix.from_pandas_edgelist(test, source="SOURCE_SUBREDDIT", target="TARGET_SUBREDDIT", edge_attr="LINK_SENTIMENT")
test_set1 = set(G_test.edges)
test_set2 = set([(pair[1], pair[0]) for pair in test_set1])
test_set = test_set1 | test_set2

In [ ]:
# df_train = pd.read_csv('train.csv')
# df_test = pd.read_csv('test.csv')
# G_train = nx.convert_matrix.from_pandas_edgelist(df_train, source="SOURCE_SUBREDDIT", target="TARGET_SUBREDDIT", edge_attr="LINK_SENTIMENT")
# G_test = nx.convert_matrix.from_pandas_edgelist(df_test, source="SOURCE_SUBREDDIT", target="TARGET_SUBREDDIT", edge_attr="LINK_SENTIMENT")
# test_set = set(G_test.edges)

In [38]:
def common_neighbor(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    score = {}
    for node_pair in all_unconnected_node_pairs:
        if node_pair[0] != node_pair[1]:
            score[node_pair] = len(set(G.neighbors(node_pair[0])) & set(G.neighbors(node_pair[1])))
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    guesses = set([guess[0] for guess in sorted_best_guesses])
    reverse = set([(guess[0][1],guess[0][0]) for guess in sorted_best_guesses])
    return guesses | reverse

def jaccard(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    j = list(nx.jaccard_coefficient(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in j:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    guesses = set([guess[0] for guess in sorted_best_guesses])
    reverse = set([(guess[0][1],guess[0][0]) for guess in sorted_best_guesses])
    return guesses | reverse

def pa(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    pa = list(nx.preferential_attachment(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in pa:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    guesses = set([guess[0] for guess in sorted_best_guesses])
    reverse = set([(guess[0][1],guess[0][0]) for guess in sorted_best_guesses])
    return guesses | reverse

def rai(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    rai = list(nx.resource_allocation_index(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in rai:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size*2:2]
    guesses = set([guess[0] for guess in sorted_best_guesses])
    reverse = set([(guess[0][1],guess[0][0]) for guess in sorted_best_guesses])
    return guesses | reverse

def aai(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    aai = list(nx.adamic_adar_index(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in aai:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    guesses = set([guess[0] for guess in sorted_best_guesses])
    reverse = set([(guess[0][1],guess[0][0]) for guess in sorted_best_guesses])
    return guesses | reverse

def random_guess(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    return set(random.sample(all_unconnected_node_pairs, size))

In [17]:
def score(G, func, size):
    guesses = func(G, size)
    precision = len(guesses & test_set) / len(guesses)
    recall = len(guesses & test_set) / len(test_set)
    return (precision, recall)

In [50]:
funcs = [common_neighbor, jaccard, pa, rai, random_guess]
func_labels = ['common neighbor', 'jaccard', 'preferential attachment', 'resource allocation index', 'random guess']
for label,func in zip(func_labels, funcs):
    print(label, score(G_train, func, 100))

common neighbor (0.8, 0.00023648402823255474)
jaccard (0.05102040816326531, 2.273884886851488e-05)
preferential attachment (0.8333333333333334, 0.0002273884886851488)
resource allocation index (0.6483516483516484, 0.00026831841664847557)
random guess (0.01, 2.273884886851488e-06)
